In [2]:
from openprompt.plms import load_plm
from openprompt.prompts import ManualVerbalizer, ManualTemplate
from openprompt.pipeline_base import PromptForClassification
from transformers import RobertaTokenizer
import torch

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('my_tokenizer')

# Load the PLM, tokenizer, and wrapper class
plm, _, _, WrapperClass = load_plm("roberta", "roberta-large")

# Define the template (recreate it as it was during training)
template = ManualTemplate(
    tokenizer=tokenizer, 
    text='{"placeholder":"text_a"} Sentence for analyze: This sentence is {"mask"}.' 
)

# Define the verbalizer (recreate it as it was during training)
verbalizer = ManualVerbalizer(
    tokenizer=tokenizer, 
    num_classes=2, 
    label_words=['positive','negative']
)

# Create the model instance
model = PromptForClassification(plm, template, verbalizer)

# Load the saved model state
model.load_state_dict(torch.load('best_model_by_template.pt'))

<All keys matched successfully>

In [7]:
import torch
from openprompt.data_utils import InputExample
from openprompt.pipeline_base import PromptDataLoader, PromptForClassification

# Define the device (use GPU if available, otherwise use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict_new_sentence(sentence, model, template, tokenizer, device, WrapperClass):
    # Create an InputExample object
    input_example = InputExample(text_a=sentence, label=0)  # Label is dummy
    
    # Create a DataLoader for the single example
    single_example_dataloader = PromptDataLoader(
        dataset=[input_example], 
        template=template, 
        tokenizer=tokenizer, 
        tokenizer_wrapper_class=WrapperClass, 
        batch_size=1, 
        shuffle=False,
        max_seq_length=128
    )

    # Move model to the appropriate device
    model.to(device)
    model.eval()

    # Forward pass: compute logits
    with torch.no_grad():
        for batch in single_example_dataloader:
            # Move batch to the appropriate device
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(batch)

    # Convert logits to probabilities
    probs = torch.nn.functional.softmax(logits, dim=1)

    # Get the class with the highest probability
    predicted_class = torch.argmax(probs, dim=1).item()

    return predicted_class

# Example usage
new_sentence = "someone is really"
predicted_class = predict_new_sentence(new_sentence, model, template, tokenizer, device, WrapperClass)
print(f"Predicted class for the new sentence: {predicted_class}")

tokenizing: 1it [00:00, 500.45it/s]


Predicted class for the new sentence: 1
